
# Post-processing Volumetric CFD Data with maia : Surface extraction and Vizualization 

## Introduction

In this tutorial, we demonstrate how to perform distributed post-processing of CFD simulation results using **maia**, a Python/C++ library for parallel CGNS tree manipulation. The input is a volumetric mesh tree dispersed across MPI processes, obtained from a CFD solver, and stored in the CGNS format.

The objective is to extract a surfacic mesh that corresponds to wall boundaries (designated by the "Walls" family), collect and combine the related physical fields (such as skin friction and pressure), and get the data ready for export or visualization.

After completing this lesson, you will be able to: 
- Use maia and `mpi4py` to load and examine a distributed CGNS mesh,
- Using family tags, extract a surface mesh from a volumetric tree,
- Consolidate dispersed field data into a single FlowSolution container,
- For post-processing tools, save the resultant data to a new CGNS file.

This practical example offers an introduction to using maia to write scalable post-processing scripts in a parallel computing environment.

## Step 1 -- import modules

Maia operates in parallel! The so-called COMM_WORLD communicator must be imported from mpi4py first because practically all functions require an MPI communicator.

We need to import others python modules like numpy, sys, and mtplotlib.

In [1]:
import numpy as np
from mpi4py import MPI
import sys
import matplotlib.pyplot as plt
comm = MPI.COMM_WORLD

Open the documentation that will be useful for this TP first: /Maia/1.3/index.html https://onera.github.io
Take a brief look at the structure of the various modules (UserManual) and the definition of parallel CGNSTree (Introducion > Maia CGNS Tree). Then, import maia and the module pytree of maia

In [2]:
import maia
import maia.pytree as PT

Step 3 -- Uncompress Tress

Cgns files are stored in this directory:

In [3]:
CASE_DIR = '/home/jcoulet/Public/maia_training/MESHES'

In [4]:
def uncompress_tree():
    """ Tree has been stored with R4 arrays and Std elements format to
    reduce file size. This function reput it in NG format """
    from pathlib import Path
    filepath = Path(CASE_DIR) / 'airplane.cgns'
    if not filepath.exists():
        if comm.rank == 0:
            print("MISSING FILE -- Regenerate case")
        elt_filepath = Path(CASE_DIR) / 'airplane_eltR4.cgns'
        tree  = maia.io.file_to_dist_tree(elt_filepath, comm)
        for array in PT.get_nodes_from_label(tree, 'DataArray_t'):
            if PT.get_value_type(array) == 'R4':
                PT.set_value(array, PT.get_value(array).astype(float))
        maia.algo.dist.convert_elements_to_ngon(tree, comm)
        maia.io.dist_tree_to_file(tree, filepath, comm)

## Step 2 -- Partitionned tree

Create a function that can :
- Load the file airplane.cgns.
- Convert the file into a distributed CGNSree (tree).
- Partition the distributed tree by calling maia.factory.partition_dist_tree, which produce a partitionned tree suitable for parralle computation. 
- Return the partitioned tree (ptree).


In [5]:
def get_solver_tree():
    uncompress_tree()
    tree  = maia.io.file_to_dist_tree(CASE_DIR + '/airplane.cgns', comm)
    ptree = maia.factory.partition_dist_tree(tree, comm, data_transfer='FIELDS')
    return ptree

## Step 3 -- Plot Field 

Create the plot_1d_profile function that:
- Has these arguments : local arrays x (positions), f (field values), the communicator comm, the label for the x-axis, and the label for the y-axis.
- Gathers the data to rank 0, where it concatenates the arrays and plots f versus x using matplotlib.
- Customize axis labels with xlabel and ylabel.
- Only rank 0 creates the plot. 

In [6]:
def plot_1d_profile(x, f, comm, xlabel='Pos', ylabel='Field'):
    x = comm.gather(x, root=0)
    f = comm.gather(f, root=0)
    if comm.Get_rank() == 0:
        # > Concatenate arrays
        x = np.concatenate(x)
        f = np.concatenate(f)

        # > Plot
        fig   = plt.figure()
        fig, axes1 = plt.subplots()
        axes1.set_xlabel(xlabel)
        axes1.set_ylabel(ylabel)
        axes1.plot(x, f, 'o')
        #plt.show() or
        fig.savefig('figure.png', dpi=300)

Input of this TP is a volumic tree, as it would come from
the solver.
Here we get it with the following function :

In [7]:
sol_tree = get_solver_tree()
PT.print_tree(sol_tree)

Distributed read of file /home/jcoulet/Public/maia_training/MESHES/airplane.cgns...
Read completed (49.97 s) -- Size of dist_tree for current rank is 552.3MiB (Σ=552.3MiB)
Partitioning tree of 1 initial block...
Partitioning completed (11.05 s) -- Nb of cells for current rank is 4.5M (Σ=4.5M)
CGNSTree CGNSTree_t 
├───CGNSLibraryVersion CGNSLibraryVersion_t R4 [4.2]
└───Base CGNSBase_t I4 [3 3]
    ├───FTF Family_t 
    │   └───FamilyBC FamilyBC_t "BCWallViscous"
    ├───Farfield Family_t 
    │   └───FamilyBC FamilyBC_t "BCFarfield"
    ├───Fuselage Family_t 
    │   └───FamilyBC FamilyBC_t "BCWallViscous"
    ├───HTail Family_t 
    │   └───FamilyBC FamilyBC_t "BCWallViscous"
    ├───Symmetry Family_t 
    │   └───FamilyBC FamilyBC_t "BCSymmetryPlane"
    ├───VTail Family_t 
    │   └───FamilyBC FamilyBC_t "BCWallViscous"
    ├───Wing Family_t 
    │   └───FamilyBC FamilyBC_t "BCWallViscous"
    ├───Walls Family_t 
    │   ├───.Solver#Output:forcesWalls UserDefinedData_t 
    │   │   

What is the kind (distributed, partitioned, full) of sol_tree ?

Now let's do some post operations.

## Step 4 -- Extraction

Extract a surfacic mesh corresponding to the faces belonging to Walls family.

In [8]:
ext_tree = maia.algo.part.extract_part_from_family(sol_tree, 'Walls', comm)
PT.print_tree(ext_tree)

CGNSTree CGNSTree_t 
├───CGNSLibraryVersion CGNSLibraryVersion_t R4 [4.2]
└───Base CGNSBase_t I4 [2 3]
    ├───Zone.P0.N0 Zone_t I4 [[ 61933 123405      0]]
    │   ├───ZoneType ZoneType_t "Unstructured"
    │   ├───:CGNS#GlobalNumbering UserDefinedData_t 
    │   │   ├───Vertex DataArray_t I4 (61933,)
    │   │   └───Cell DataArray_t I4 (123405,)
    │   ├───GridCoordinates GridCoordinates_t 
    │   │   ├───CoordinateX DataArray_t R8 (61933,)
    │   │   ├───CoordinateY DataArray_t R8 (61933,)
    │   │   └───CoordinateZ DataArray_t R8 (61933,)
    │   ├───NGonElements Elements_t I4 [22  0]
    │   │   ├───ElementRange IndexRange_t I4 [     1 123405]
    │   │   ├───ElementStartOffset DataArray_t I4 (123406,)
    │   │   ├───ElementConnectivity DataArray_t I4 (370215,)
    │   │   └───:CGNS#GlobalNumbering UserDefinedData_t 
    │   │       └───Element DataArray_t I4 (123405,)
    │   ├───ZoneBC ZoneBC_t 
    │   │   ├───FTF BC_t "FamilySpecified"
    │   │   │   ├───GridLocation Gri

NB : output fields are stored as several ZSR (one per extraction)
If we want to visualize it in Paraview we have to merge it into
a single FlowSolution container.

To do that follow these steps:
- Get the list of fields to create
- Create empty arrays of size nb_cell
- Loop over ZSR and fill arrays:
- Global_array[zsr_point_list] = zsr_value
- Remove ZSR

In [9]:
for zone in PT.get_all_Zone_t(ext_tree):
    zsr_list = PT.get_children_from_label(zone, 'ZoneSubRegion_t')
    sizes = [PT.Subset.n_elem(zsr) for zsr in zsr_list]

    fields = [PT.get_name(n) for n in 
                PT.get_children_from_label(zsr_list[0], 'DataArray_t')]

    merged_fields = {field: np.empty(sum(sizes)) for field in fields}

    for zsr in zsr_list:
        pl = PT.get_child_from_name(zsr, 'PointList')[1][0]
        for fld_node in PT.get_children_from_label(zsr, 'DataArray_t'):
            merged_fields[PT.get_name(fld_node)][pl-1] = PT.get_value(fld_node)

    PT.new_FlowSolution('ExtractedSol', loc='CellCenter', fields=merged_fields, parent=zone)
    PT.rm_children_from_label(zone, 'ZoneSubRegion_t')
PT.print_tree(ext_tree)

CGNSTree CGNSTree_t 
├───CGNSLibraryVersion CGNSLibraryVersion_t R4 [4.2]
└───Base CGNSBase_t I4 [2 3]
    ├───Zone.P0.N0 Zone_t I4 [[ 61933 123405      0]]
    │   ├───ZoneType ZoneType_t "Unstructured"
    │   ├───:CGNS#GlobalNumbering UserDefinedData_t 
    │   │   ├───Vertex DataArray_t I4 (61933,)
    │   │   └───Cell DataArray_t I4 (123405,)
    │   ├───GridCoordinates GridCoordinates_t 
    │   │   ├───CoordinateX DataArray_t R8 (61933,)
    │   │   ├───CoordinateY DataArray_t R8 (61933,)
    │   │   └───CoordinateZ DataArray_t R8 (61933,)
    │   ├───NGonElements Elements_t I4 [22  0]
    │   │   ├───ElementRange IndexRange_t I4 [     1 123405]
    │   │   ├───ElementStartOffset DataArray_t I4 (123406,)
    │   │   ├───ElementConnectivity DataArray_t I4 (370215,)
    │   │   └───:CGNS#GlobalNumbering UserDefinedData_t 
    │   │       └───Element DataArray_t I4 (123405,)
    │   ├───ZoneBC ZoneBC_t 
    │   │   ├───FTF BC_t "FamilySpecified"
    │   │   │   ├───GridLocation Gri

What kind of tree is the result of the extraction ? 
How can we save it as a single merged file with dist_tree_to_file ?

In [10]:
dext_tree = maia.factory.recover_dist_tree(ext_tree, comm)
maia.io.dist_tree_to_file(dext_tree, 'Walls.cgns', comm)
PT.print_tree(dext_tree)

Distributed write of a 3.8MiB dist_tree (Σ=3.8MiB)...
CGNSTree CGNSTree_t 
Write completed [Walls.cgns] (0.77 s)
├───CGNSLibraryVersion CGNSLibraryVersion_t R4 [4.2]
└───Base CGNSBase_t I4 [2 3]
    ├───FTF Family_t 
    │   └───FamilyBC FamilyBC_t "BCWallViscous"
    ├───Fuselage Family_t 
    │   └───FamilyBC FamilyBC_t "BCWallViscous"
    ├───HTail Family_t 
    │   └───FamilyBC FamilyBC_t "BCWallViscous"
    ├───VTail Family_t 
    │   └───FamilyBC FamilyBC_t "BCWallViscous"
    ├───Walls Family_t 
    │   ├───.Solver#Output:forcesWalls UserDefinedData_t 
    │   │   ├───var DataArray_t "flux_rou [...]_row"
    │   │   ├───pinf DataArray_t R8 [101325.296875]
    │   │   ├───fluxcoeff DataArray_t R8 [0.00051035]
    │   │   ├───torquecoeff DataArray_t R8 [0.00121421]
    │   │   ├───xtorque DataArray_t R8 [2.02067161]
    │   │   ├───ytorque DataArray_t R8 [0.]
    │   │   ├───ztorque DataArray_t R8 [0.2711958]
    │   │   └───loc DataArray_t "interface"
    │   └───.Solver#Output:S

## Step 5 -- Slicing

Compute a slice of the volumic tree using the plane equation y = 1
Use the same procedure as above to save the slice (in a other file)  and visualize it.

In [11]:
slice_tree = maia.algo.part.plane_slice(sol_tree, [0.,1.,0.,1], comm)
dslice_tree = maia.factory.recover_dist_tree(slice_tree, comm)
sys.stdout.flush()
PT.print_tree(dslice_tree)

Plane slice completed (1.62 s)
CGNSTree CGNSTree_t 
├───CGNSLibraryVersion CGNSLibraryVersion_t R4 [4.2]
└───Base CGNSBase_t I4 [2 3]
    └───Zone Zone_t I4 [[ 85248 169976      0]]
        ├───ZoneType ZoneType_t "Unstructured"
        ├───FamilyName FamilyName_t "Unspecified"
        ├───:CGNS#Distribution UserDefinedData_t 
        │   ├───Vertex DataArray_t I4 [    0 85248 85248]
        │   └───Cell DataArray_t I4 [     0 169976 169976]
        ├───GridCoordinates GridCoordinates_t 
        │   ├───CoordinateX DataArray_t R8 (85248,)
        │   ├───CoordinateY DataArray_t R8 (85248,)
        │   └───CoordinateZ DataArray_t R8 (85248,)
        ├───TRI_3 Elements_t I4 [5 0]
        │   ├───ElementRange IndexRange_t I4 [     1 169976]
        │   ├───ElementConnectivity DataArray_t I4 (509928,)
        │   └───:CGNS#Distribution UserDefinedData_t 
        │       └───Element DataArray_t I4 [     0 169976 169976]
        ├───BAR_2 Elements_t I4 [3 0]
        │   ├───ElementRange Inde

Save the dist_tree into file called Slice.cgns with dist_tree_to_file.

In [12]:
maia.io.dist_tree_to_file(dslice_tree, 'Slice.cgns', comm)

Distributed write of a 3.9MiB dist_tree (Σ=3.9MiB)...
Write completed [Slice.cgns] (0.71 s)


Now, update the slice creation to transfer the fields computed in
the volumic tree to the slice.

In [13]:
slice_tree = maia.algo.part.plane_slice(sol_tree, [0.,1.,0.,1], comm, containers_name=['FlowSolution#Centers'])
dslice_tree = maia.factory.recover_dist_tree(slice_tree, comm)
PT.print_tree(dslice_tree)

Plane slice completed (2.22 s)
CGNSTree CGNSTree_t 
├───CGNSLibraryVersion CGNSLibraryVersion_t R4 [4.2]
└───Base CGNSBase_t I4 [2 3]
    └───Zone Zone_t I4 [[ 85248 169976      0]]
        ├───ZoneType ZoneType_t "Unstructured"
        ├───FamilyName FamilyName_t "Unspecified"
        ├───:CGNS#Distribution UserDefinedData_t 
        │   ├───Vertex DataArray_t I4 [    0 85248 85248]
        │   └───Cell DataArray_t I4 [     0 169976 169976]
        ├───GridCoordinates GridCoordinates_t 
        │   ├───CoordinateX DataArray_t R8 (85248,)
        │   ├───CoordinateY DataArray_t R8 (85248,)
        │   └───CoordinateZ DataArray_t R8 (85248,)
        ├───TRI_3 Elements_t I4 [5 0]
        │   ├───ElementRange IndexRange_t I4 [     1 169976]
        │   ├───ElementConnectivity DataArray_t I4 (509928,)
        │   └───:CGNS#Distribution UserDefinedData_t 
        │       └───Element DataArray_t I4 [     0 169976 169976]
        ├───BAR_2 Elements_t I4 [3 0]
        │   ├───ElementRange Inde

Save the modified dist_tree into the same file Slice.cgns.

In [14]:
maia.io.dist_tree_to_file(dslice_tree, 'Slice.cgns', comm)

Distributed write of a 3.9MiB dist_tree (Σ=3.9MiB)...
Write completed [Slice.cgns] (0.64 s)


## Step 6 -- Plot a 1d profile

In this last step, we are going to update again our slice in order
to plot a 1d profile $Pressure = f(x)$ around the Wing.

Extracted fields on wing are stored in BCDataSet Wing/BCDataSet:Skin/NeumannData,
but slice function can only transfer data found in FlowSolution or ZSR
so we need to move the fields in a ZSR container
This is done by the above snippet : 

In [15]:
zone = PT.get_node_from_label(sol_tree, 'Zone_t')
bc_wing = PT.get_node_from_name_and_label(sol_tree, 'Wing', 'BC_t')
if bc_wing is not None:
    bc_wing_dataset = PT.get_node_from_label(bc_wing, 'BCData_t')
    bc_wing_fields = {PT.get_name(n) : PT.get_value(n) for n in 
                    PT.get_children_from_label(bc_wing_dataset, 'DataArray_t')}

    zsr_wing = PT.new_ZoneSubRegion('ZSR_Wing', bc_name="Wing", fields=bc_wing_fields, parent=zone)
PT.print_tree(sol_tree)

CGNSTree CGNSTree_t 
├───CGNSLibraryVersion CGNSLibraryVersion_t R4 [4.2]
└───Base CGNSBase_t I4 [3 3]
    ├───FTF Family_t 
    │   └───FamilyBC FamilyBC_t "BCWallViscous"
    ├───Farfield Family_t 
    │   └───FamilyBC FamilyBC_t "BCFarfield"
    ├───Fuselage Family_t 
    │   └───FamilyBC FamilyBC_t "BCWallViscous"
    ├───HTail Family_t 
    │   └───FamilyBC FamilyBC_t "BCWallViscous"
    ├───Symmetry Family_t 
    │   └───FamilyBC FamilyBC_t "BCSymmetryPlane"
    ├───VTail Family_t 
    │   └───FamilyBC FamilyBC_t "BCWallViscous"
    ├───Wing Family_t 
    │   └───FamilyBC FamilyBC_t "BCWallViscous"
    ├───Walls Family_t 
    │   ├───.Solver#Output:forcesWalls UserDefinedData_t 
    │   │   ├───var DataArray_t "flux_rou [...]_row"
    │   │   ├───pinf DataArray_t R8 [101325.296875]
    │   │   ├───fluxcoeff DataArray_t R8 [0.00051035]
    │   │   ├───torquecoeff DataArray_t R8 [0.00121421]
    │   │   ├───xtorque DataArray_t R8 [2.02067161]
    │   │   ├───ytorque DataArray_t R8 

Update the slice to get the created ZSR, in addition to the volumic fields.
What is the location of ZSR_Wing on output mesh ?

In [16]:
slice_tree = maia.algo.part.plane_slice(sol_tree, [0.,1.,0.,1], comm, containers_name=['ZSR_Wing', 'FlowSolution#Centers'])
PT.print_tree(slice_tree)


CGNSTree CGNSTree_t 
Plane slice completed (3.65 s)
├───CGNSLibraryVersion CGNSLibraryVersion_t R4 [4.2]
└───Base CGNSBase_t I4 [2 3]
    └───Zone.P0.N0 Zone_t I4 [[ 85248 169976      0]]
        ├───ZoneType ZoneType_t "Unstructured"
        ├───GridCoordinates GridCoordinates_t 
        │   ├───CoordinateX DataArray_t R8 (85248,)
        │   ├───CoordinateY DataArray_t R8 (85248,)
        │   └───CoordinateZ DataArray_t R8 (85248,)
        ├───TRI_3 Elements_t I4 [5 0]
        │   ├───ElementRange IndexRange_t I4 [     1 169976]
        │   ├───ElementConnectivity DataArray_t I4 (509928,)
        │   └───:CGNS#GlobalNumbering UserDefinedData_t 
        │       ├───Element DataArray_t I4 (169976,)
        │       └───Sections DataArray_t I4 (169976,)
        ├───BAR_2 Elements_t I4 [3 0]
        │   ├───ElementRange IndexRange_t I8 [169977 170496]
        │   ├───ElementConnectivity DataArray_t I4 (1040,)
        │   └───:CGNS#GlobalNumbering UserDefinedData_t 
        │       ├───Ele

To plot our profile, we need to compute the corresponding X center of each generated ZSR_Wing element : 
- Compute the relevevant centers in the output sliced zone.
- Get the PointList of elements belonging to ZSR_Wing.
- Use it to access centers arrays to extract X coordinate (be careful, pl must be shifted to start at 0).
- Get the pressure field in the ZSR.

NB, some ranks may not know the ZSR_Wing at all. Create empty arrays in this case.

In [17]:
slice_zone = PT.get_node_from_label(slice_tree, 'Zone_t')
zsr_n = PT.get_child_from_name(slice_zone, 'ZSR_Wing')
PT.print_tree(slice_zone)
if zsr_n is not None:
    edge_center = maia.algo.geometry.compute_elements_center(slice_zone, 1)
    # > Get edge offset
    is_edge = lambda n: PT.get_label(n)=='Elements_t' and PT.Element.CGNSName(n)=='BAR_2'
    edge_n = PT.get_child_from_predicate(slice_zone, is_edge)
    er_offset = PT.Element.Range(edge_n)[0]
    zsr_idx = PT.get_child_from_name(zsr_n, 'PointList')[1][0] - er_offset
    if edge_center is not None:
        cx = edge_center[0::3][zsr_idx]
        pressure = PT.get_value(PT.get_child_from_name(zsr_n, 'Pressure'))
else :
    cx = np.empty(0, dtype=np.float64)
    pressure = np.empty(0, dtype=np.float64)
    
    # Finaly, you can send the data to plot_1d_profile function and view the created png file
    plot_1d_profile(cx, pressure, comm, xlabel='X', ylabel='Pressure')

Zone.P0.N0 Zone_t I4 [[ 85248 169976      0]]
├───ZoneType ZoneType_t "Unstructured"
├───GridCoordinates GridCoordinates_t 
│   ├───CoordinateX DataArray_t R8 (85248,)
│   ├───CoordinateY DataArray_t R8 (85248,)
│   └───CoordinateZ DataArray_t R8 (85248,)
├───TRI_3 Elements_t I4 [5 0]
│   ├───ElementRange IndexRange_t I4 [     1 169976]
│   ├───ElementConnectivity DataArray_t I4 (509928,)
│   └───:CGNS#GlobalNumbering UserDefinedData_t 
│       ├───Element DataArray_t I4 (169976,)
│       └───Sections DataArray_t I4 (169976,)
├───BAR_2 Elements_t I4 [3 0]
│   ├───ElementRange IndexRange_t I8 [169977 170496]
│   ├───ElementConnectivity DataArray_t I4 (1040,)
│   └───:CGNS#GlobalNumbering UserDefinedData_t 
│       ├───Element DataArray_t I4 (520,)
│       └───Sections DataArray_t I4 (520,)
├───ZoneBC ZoneBC_t 
│   ├───Farfield BC_t "Null"
│   │   ├───GridLocation GridLocation_t "EdgeCenter"
│   │   ├───PointList IndexArray_t I4 (1, 58)
│   │   └───:CGNS#GlobalNumbering UserDefinedData_t